<img src="../img/mCIDaeNnb.png" alt="Logo CiDAEN" align="right">

<br><br><br>
<h2><font color="#00586D" size=4>Trabajo Fin de Máster</font></h2>

<h1><font color="#00586D" size=5>Análisis y Predicción de Resultados en Partidas de Clash Royale:<br><b>1. Adquisición de Datos</b></font></h1>
<br><br><br>


<div align="right">
<font color="#00586D" size=3>Máster en Ciencia de Datos e Ingeniería de Datos en la Nube</font><br>
<font color="#00586D" size=3>Universidad de Castilla-La Mancha</font><br>
</div>

<font color="#00586D" size=3>Iván Fernández García</font><br>
<font color="#00586D" size=3>Curso académico 2024/2025</font><br>

---

<a id="indice"></a>
<h2><font color="#00586D" size=5>Índice</font></h2>


* [1. Introducción](#section1)
* [2. Clash Royale API](#section2)
    * [2.1. Cartas](#section2_1)
    * [2.2. Clanes](#section2_2)
    * [2.3. Jugadores](#section2_3)
    * [2.4. Otros](#section2_4)
* [3. Metodología](#section3)
* [4. Adquisición y almacenamiento](#section4)
    * [4.1. Cartas](#section4_1)
    * [4.2. Partidas](#section4_2)
* [5. Conclusiones](#section5)

---

In [1]:
import os
import json
import random
import requests

---

<a id="section1"></a>
## <font color="#00586D"> 1. Introducción</font>

Esta primera fase se centrará exclusivamente en la adquisición y recopilación de datos relevantes para resolver el problema que queremos abordar. Para ello, utilizaremos principalmente la API oficial de Clash Royale, la cual proporciona acceso a información detallada sobre jugadores, clanes, partidas recientes, estadísticas de cartas, etc. Además, exploraremos la posibilidad de complementar esta información con datos provenientes de otras fuentes.

El resultado de esta etapa será una gran cantidad de datos en crudo, todavía sin ser procesados, que servirán como base para las siguientes fases del proyecto.

---

<a id="section2"></a>
## <font color="#00586D"> 2. Clash Royale API</font>

La API oficial de Clash Royale permite acceder de forma programática a una amplia variedad de datos del juego. Su uso requiere autenticación mediante un *token* de acceso, el cual se puede obtener registrando una cuenta en el [portal para desarrolladores de Supercell](https://developer.clashroyale.com/#/). Gracias a este servicio, es posible construir herramientas que utilizan la información para proporcionar valor a la comunidad.

En primer lugar, vamos a acceder a la API utilizando la URL correspondiente y el *token* que hemos generado:


In [2]:
BASE_URL = "https://api.clashroyale.com/v1"
TOKEN = os.getenv("CR_API_TOKEN")
headers = {"Content-type": "application/json", "Authorization": "Bearer " + TOKEN}

A continuación se van a explicar y a probar algunos de los *endpoints* más comunes para extraer información del juego.

<a id="section2_1"></a>
### <font color="#00586D"> 2.1. Cartas</font>

Para obtener información de las cartas podemos utilizar el *endpoint* `/cards`.

In [3]:
def get_cards_info():
    url = BASE_URL + "/cards"
    response = requests.get(url, headers=headers)
    return response.json()

In [4]:
cards = get_cards_info()
cards.keys()

dict_keys(['items', 'supportItems'])

Vamos a obtener la información de las diez primeras cartas:

In [5]:
cards["items"][:10]

[{'name': 'Knight',
  'id': 26000000,
  'maxLevel': 14,
  'maxEvolutionLevel': 1,
  'elixirCost': 3,
  'iconUrls': {'medium': 'https://api-assets.clashroyale.com/cards/300/jAj1Q5rclXxU9kVImGqSJxa4wEMfEhvwNQ_4jiGUuqg.png',
   'evolutionMedium': 'https://api-assets.clashroyale.com/cardevolutions/300/jAj1Q5rclXxU9kVImGqSJxa4wEMfEhvwNQ_4jiGUuqg.png'},
  'rarity': 'common'},
 {'name': 'Archers',
  'id': 26000001,
  'maxLevel': 14,
  'maxEvolutionLevel': 1,
  'elixirCost': 3,
  'iconUrls': {'medium': 'https://api-assets.clashroyale.com/cards/300/W4Hmp8MTSdXANN8KdblbtHwtsbt0o749BbxNqmJYfA8.png',
   'evolutionMedium': 'https://api-assets.clashroyale.com/cardevolutions/300/W4Hmp8MTSdXANN8KdblbtHwtsbt0o749BbxNqmJYfA8.png'},
  'rarity': 'common'},
 {'name': 'Goblins',
  'id': 26000002,
  'maxLevel': 14,
  'elixirCost': 2,
  'iconUrls': {'medium': 'https://api-assets.clashroyale.com/cards/300/X_DQUye_OaS3QN6VC9CPw05Fit7wvSm3XegXIXKP--0.png'},
  'rarity': 'common'},
 {'name': 'Giant',
  'id': 26000

Cada uno de ellas contiene los siguientes atributos:
* `name`: Nombre de la carta.
* `id`: Identificador de la carta.
* `maxLevel`: Nivel máximo de la carta. Depende de la rareza, pero realmente dentro del juego el nivel máximo de todas las cartas es el 14 (+1).
* `maxEvolutionLevel`: Nivel máximo de la evolución de la carta, el cual solo aparece en las cartas con evolución y siempre es 1.
* `elixirCost`: Coste de elixir de la carta.
* `iconUrls`: Iconos
    * `medium`: Icono de la carta.
    * `evolutionMedium`: Icono de la evolución de la carta, el cual solo aparece en las cartas con evolución.
* `rarity`: Rareza de la carta. Puede tomar los valores *common*, *rare*, *epic*, *legendary* y *champion*.

También tenemos datos sobre las tropas de las torres de coronas, conocidas como cartas de soporte.

In [6]:
cards["supportItems"]

[{'name': 'Tower Princess',
  'id': 159000000,
  'maxLevel': 14,
  'iconUrls': {'medium': 'https://api-assets.clashroyale.com/cards/300/Nzo5Gjbh7NG6O3Hyu7ev54Pu5zK7vDMR2fbpGdVsS64.png'},
  'rarity': 'common'},
 {'name': 'Cannoneer',
  'id': 159000001,
  'maxLevel': 9,
  'iconUrls': {'medium': 'https://api-assets.clashroyale.com/cards/300/cUfU4UowRdbIiRvxv0ns4ezQUNndJTy7D2q4I_K_fzg.png'},
  'rarity': 'epic'},
 {'name': 'Dagger Duchess',
  'id': 159000002,
  'maxLevel': 6,
  'iconUrls': {'medium': 'https://api-assets.clashroyale.com/cards/300/MVj028nMLCmBuP3HlV503uxVAxFg7jyliJVZ5JYJ1h8.png'},
  'rarity': 'legendary'},
 {'name': 'Royal Chef',
  'id': 159000004,
  'maxLevel': 6,
  'iconUrls': {'medium': 'https://api-assets.clashroyale.com/cards/300/CLxP2o0iX7q9peMkekI8Ki4DgWixra8L5aUbiAeBU9U.png'},
  'rarity': 'legendary'}]

Estas no tienen cuestan elixir ni tienen evoluciones.

<a id="section2_2"></a>
### <font color="#00586D"> 2.2. Clanes</font>

En relación con los clanes, hay diferentes *endpoints* de interés.

Por ejemplo, podemos obtener información sobre un conjunto de clanes (tag, nombre, ubición, número de miembros...) a partir de distintos parámetros con `/clans`:

In [7]:
def get_clans(params=None):
    url = BASE_URL + "/clans"
    response = requests.get(url, headers=headers, params=params)
    return response.json()

In [8]:
params = {
    "name": "Royale",
    "locationId": 57000003,
    "minMembers": 10,
    "maxMembers": 40,
    "minScore": 1500,
    "limit": 10
}

clans = get_clans(params)
clans["items"]

[{'tag': '#PJQ0RJL8',
  'name': 'ROYALE',
  'type': 'open',
  'badgeId': 16000013,
  'clanScore': 66861,
  'clanWarTrophies': 1643,
  'location': {'id': 57000003, 'name': 'Asia', 'isCountry': False},
  'requiredTrophies': 3000,
  'donationsPerWeek': 70,
  'clanChestLevel': 1,
  'clanChestMaxLevel': 0,
  'members': 35},
 {'tag': '#GYCU0PQV',
  'name': 'ŁČ ROYALE ĞÂŇĢ',
  'type': 'inviteOnly',
  'badgeId': 16000163,
  'clanScore': 49498,
  'clanWarTrophies': 771,
  'location': {'id': 57000003, 'name': 'Asia', 'isCountry': False},
  'requiredTrophies': 0,
  'donationsPerWeek': 514,
  'clanChestLevel': 1,
  'clanChestMaxLevel': 0,
  'members': 20},
 {'tag': '#QUYLQ89Q',
  'name': 'mahed royale',
  'type': 'inviteOnly',
  'badgeId': 16000000,
  'clanScore': 26688,
  'clanWarTrophies': 0,
  'location': {'id': 57000003, 'name': 'Asia', 'isCountry': False},
  'requiredTrophies': 0,
  'donationsPerWeek': 16,
  'clanChestLevel': 1,
  'clanChestMaxLevel': 0,
  'members': 15},
 {'tag': '#QCLR0Y82'

También podemos acceder a esta información para un clan en concreto a través de su tag utilizando el *endpoint* `/clans/{clanTag}`, aunque en este caso también aparece una lista con información sobre los miembros del clan (tag, nombre, rol, donaciones...).

In [9]:
def get_clan_info(tag):
    formatted_tag = tag.replace("#", "")
    url = BASE_URL + "/clans/%23" + formatted_tag
    response = requests.get(url, headers=headers)
    return response.json()

In [10]:
clan_tag = "#G9PVLCL8"
get_clan_info(clan_tag)

{'tag': '#G9PVLCL8',
 'name': 'Royale',
 'type': 'open',
 'description': '*Aktiflik esas alınır**Klan savaşları yapmayan atılır* Küfür yasaktır*. *Bağış yapılır*',
 'badgeId': 16000124,
 'clanScore': 32281,
 'clanWarTrophies': 1276,
 'location': {'id': 57000239,
  'name': 'Turkey',
  'isCountry': True,
  'countryCode': 'TR'},
 'requiredTrophies': 6000,
 'donationsPerWeek': 27,
 'clanChestStatus': 'inactive',
 'clanChestLevel': 1,
 'clanChestMaxLevel': 0,
 'members': 10,
 'memberList': [{'tag': '#YJCPJPJCG',
   'name': 'Babizi',
   'role': 'leader',
   'lastSeen': '20250413T150501.000Z',
   'expLevel': 53,
   'trophies': 8666,
   'arena': {'id': 54000019, 'name': 'PANCAKES!'},
   'clanRank': 1,
   'previousClanRank': 1,
   'donations': 0,
   'donationsReceived': 10,
   'clanChestPoints': 0},
  {'tag': '#J9U00PRVL',
   'name': 'liangshi122',
   'role': 'coLeader',
   'lastSeen': '20250415T114256.000Z',
   'expLevel': 37,
   'trophies': 7233,
   'arena': {'id': 54000016, 'name': 'Dragon S

Es posible acceder a esta lista de miembros directamente con `clans/{clanTag}/members`:

In [11]:
def get_clan_members(tag, params=None):
    formatted_tag = tag.replace("#", "")
    url = BASE_URL + "/clans/%23" + formatted_tag + "/members"
    response = requests.get(url, headers=headers, params=params)
    return response.json()

In [12]:
clan_tag = "#G9PVLCL8"
params = {"limit": 10}
clan_members = get_clan_members(clan_tag, params)
clan_members["items"]

[{'tag': '#YJCPJPJCG',
  'name': 'Babizi',
  'role': 'leader',
  'lastSeen': '20250413T150501.000Z',
  'expLevel': 53,
  'trophies': 8666,
  'arena': {'id': 54000019, 'name': 'PANCAKES!'},
  'clanRank': 1,
  'previousClanRank': 1,
  'donations': 0,
  'donationsReceived': 10,
  'clanChestPoints': 0},
 {'tag': '#J9U00PRVL',
  'name': 'liangshi122',
  'role': 'coLeader',
  'lastSeen': '20250415T114256.000Z',
  'expLevel': 37,
  'trophies': 7233,
  'arena': {'id': 54000016, 'name': 'Dragon Spa'},
  'clanRank': 2,
  'previousClanRank': 2,
  'donations': 9,
  'donationsReceived': 0,
  'clanChestPoints': 0},
 {'tag': '#J9L8PRL0L',
  'name': 'emirhan',
  'role': 'elder',
  'lastSeen': '20250411T220845.000Z',
  'expLevel': 37,
  'trophies': 7000,
  'arena': {'id': 54000016, 'name': 'Dragon Spa'},
  'clanRank': 3,
  'previousClanRank': 3,
  'donations': 0,
  'donationsReceived': 0,
  'clanChestPoints': 0},
 {'tag': '#LQRR889UP',
  'name': '亗Ａｒｈａｔ亗',
  'role': 'coLeader',
  'lastSeen': '20250414T

<a id="section2_3"></a>
### <font color="#00586D"> 2.3. Jugadores</font>

También se puede obtener mucha información sobre los jugadores, incluyendo sus últimas partidas.

Para obtener datos para un jugador concreto, se utiliza `/players/{playerTag}`:

In [13]:
def get_player_info(tag):
    formatted_tag = tag.replace("#", "")
    url = BASE_URL + "/players/%23" + formatted_tag
    response = requests.get(url, headers=headers)
    return response.json()

In [14]:
player_tag = "#202GLGUU"
get_player_info(player_tag)

{'tag': '#202GLGUU',
 'name': 'KILLER',
 'expLevel': 41,
 'trophies': 7757,
 'bestTrophies': 7839,
 'wins': 5311,
 'losses': 4558,
 'battleCount': 9869,
 'threeCrownWins': 1697,
 'challengeCardsWon': 12183,
 'challengeMaxWins': 12,
 'tournamentCardsWon': 1423,
 'tournamentBattleCount': 2798,
 'donations': 0,
 'donationsReceived': 0,
 'totalDonations': 53032,
 'warDayWins': 2,
 'clanCardsCollected': 1600,
 'arena': {'id': 54000017, 'name': 'Boot Camp'},
 'leagueStatistics': {'currentSeason': {'trophies': 7757},
  'previousSeason': {'id': '2025-03', 'trophies': 5293, 'bestTrophies': 5396},
  'bestSeason': {'id': '2017-02', 'rank': 480024, 'trophies': 4015}},
 'badges': [{'name': 'Classic12Wins',
   'level': 1,
   'maxLevel': 8,
   'progress': 2,
   'target': 10,
   'iconUrls': {'large': 'https://api-assets.clashroyale.com/playerbadges/512/eQU06dDRaTL6uIgiZiSVaOj8Y0sIUA_0Mf6Mnnr4D6Q.png'}},
  {'name': 'Grand12Wins',
   'level': 1,
   'maxLevel': 8,
   'progress': 1,
   'target': 10,
   'i

El resultado es un diccionaio con una enorme cantidad de campos, incluyendo información anidada.

Entre ellos están el tag, el nombre, el nivel y los puntos de experiencia, los trofeos actuales, el récord de trofeos, o el número de victorias y derrotas. Además, algunos valores corresponden a diccionarios que contienen mucha más información sobre las cartas del jugador (con todos los atributos comentados anteriormente), las insignias (maestrías de cartas, modos de juego...) o los logros, entre otros.

Para obtener información de partidas, podemos recurrir al registro de batalla de un jugador utilizando el *endpoint* `/players{playerTag}/battlelog`:

In [15]:
def get_player_battle_log(tag):
    formatted_tag = tag.replace("#", "")
    url = BASE_URL + "/players/%23" + formatted_tag + "/battlelog"
    response = requests.get(url, headers=headers)
    return response.json()

De todas las partidas devueltas, vamos a mostrar la información de la primera.

In [16]:
player_tag = "#202GLGUU"
battle_log = get_player_battle_log(player_tag)
battle_log[0]

{'type': 'PvP',
 'battleTime': '20250402T204141.000Z',
 'isLadderTournament': False,
 'arena': {'id': 54000017, 'name': 'Boot Camp'},
 'gameMode': {'id': 72000006, 'name': 'Ladder'},
 'deckSelection': 'collection',
 'team': [{'tag': '#202GLGUU',
   'name': 'KILLER',
   'startingTrophies': 7728,
   'trophyChange': 29,
   'crowns': 3,
   'kingTowerHitPoints': 5832,
   'princessTowersHitPoints': [3397, 3557],
   'cards': [{'name': 'P.E.K.K.A',
     'id': 26000004,
     'level': 9,
     'starLevel': 2,
     'maxLevel': 9,
     'maxEvolutionLevel': 1,
     'rarity': 'epic',
     'elixirCost': 7,
     'iconUrls': {'medium': 'https://api-assets.clashroyale.com/cards/300/MlArURKhn_zWAZY-Xj1qIRKLVKquarG25BXDjUQajNs.png',
      'evolutionMedium': 'https://api-assets.clashroyale.com/cardevolutions/300/MlArURKhn_zWAZY-Xj1qIRKLVKquarG25BXDjUQajNs.png'}},
    {'name': 'Firecracker',
     'id': 26000064,
     'level': 12,
     'maxLevel': 14,
     'maxEvolutionLevel': 1,
     'rarity': 'common',
    

Como el trabajo consiste en analizar resultados de partidas y crear modelos de aprendizaje automático capaces de predecir el ganador de las mismas, es fundamental comprender estos datos.

Como se comentará más adelante, nos enfocaremos en partidas 1vs1 (un único elemento en las listas `team` y `opponent`) de *Camino de Trofeos* (modo de juego `Ladder`). Este tipo de registro tiene el siguiente formato:

* `type`: Tipo de la partida.
* `battleTime`: Instante de tiempo de la partida.
* `isLadderTournament`: Si la partida pertenece a un torneo con reglas de subida de copas o no.
* `arena`: Arena en la que se ha jugado la partida.
    * `id`: identificador.
    * `name`: Nombre.
* `gameMode`: Modo de juego de la partida.
    * `id`: identificador.
    * `name`: Nombre.
* `deckSelection`: Forma de elegir los mazos para la partida.
* `team`: Representa el primer jugador, que tiene los atributos indentados a continuación.
    * `tag`: Tag del jugador.
    * `name`: Nombre del jugador.
    * `startingTrophies`: Número de trofeos con los que el jugador comenzó la partida.
    * `trophyChange`: Un valor positivo significa victoria y un valor negativo significa derrota.
    * `crowns`: Número de coronas del jugador en la partida.
    * `kingTowerHitPoints`: Daño producido a la torre del rey rival en la partida.
    * `princessTowerHitPoints`: Daño producido a las torres de coronas rivales en la partida.
    * `cards`: Mazo utilizado por el jugador. Cada una de las cartas utilizadas tiene los atributos indentados a continuación.
        * `name`: Nombre de la carta.
        * `id`: Identificador de la carta.
        * `level`: Nivel de la carta en el momento de la partida.
        * `evolutionLevel`: Nivel de evolución de la carta en el momento de la partida. Aparece cuando la carta tiene evolución y el jugador dispone de ella.
        * `starLevel`: Nivel de estrellas de la carta en el momento de la partida.
        * `maxLevel`: Nivel máximo de la carta. Depende de la rareza, pero realmente dentro del juego el nivel máximo de todas las cartas es el 14 (+1).
        * `maxEvolutionLevel`: Nivel de evolución de la carta en el momento de la partida. Aparece cuando la carta tiene evolución y siempre es 1.
        * `rarity`: Rareza de la carta. Puede tomar los valores *common*, *rare*, *epic*, *legendary* y *champion*.
        * `elixirCost`: Coste de elixir de la carta.
        * `iconUrls`: Iconos
            * `medium`: Icono de la carta.
            * `evolutionMedium`: Icono de la evolución de la carta, el cual solo aparece en las cartas con evolución.
    * `supportCards`: Tropa de las torres utilizada por el jugador. Es una única carta con los atributos indentados a continuación.
        * `name`: Nombre de la carta de soporte.
        * `id`: Identificador de la carta de soporte.
        * `level`: Nivel de la carta de soporte en el momento de la partida.
        * `rarity`: Rareza de la carta de soporte. Puede tomar los valores *common*, *rare*, *epic*, *legendary* y *champion*.
        * `iconUrls`: Iconos
            * `medium`: Icono de la carta.
    * `globalRank`: Ranking global del jugador.
    * `elixirLeaked`: Elixir "malgastado" durante la partida.
* `opponent`: Representa el segundo jugador, que tiene los mismos campos que el primero (contenido en `team`).
* `isHostedMatch`: Si la partida es anfitriona de un evento.
* `leagueNumber`: Número de liga competitiva de la partida.

<a id="section2_4"></a>
### <font color="#00586D"> 2.4. Otros</font>

Además de los ya mencionados, existen otros *endpoints* relacionados con ubicaciones, torneos o tablas de clasificación.

Por ejemplo, más adelante accederemos a las ubicaciones con `/locations`:

In [17]:
def get_locations():
    url = BASE_URL + "/locations"
    response = requests.get(url, headers=headers)
    return response.json()

In [18]:
locations = get_locations()
locations["items"][:10]

[{'id': 57000000, 'name': 'Europe', 'isCountry': False},
 {'id': 57000001, 'name': 'North America', 'isCountry': False},
 {'id': 57000002, 'name': 'South America', 'isCountry': False},
 {'id': 57000003, 'name': 'Asia', 'isCountry': False},
 {'id': 57000004, 'name': 'Oceania', 'isCountry': False},
 {'id': 57000005, 'name': 'Africa', 'isCountry': False},
 {'id': 57000006, 'name': 'International', 'isCountry': False},
 {'id': 57000007,
  'name': 'Afghanistan',
  'isCountry': True,
  'countryCode': 'AF'},
 {'id': 57000008,
  'name': 'Åland Islands',
  'isCountry': True,
  'countryCode': 'AX'},
 {'id': 57000009, 'name': 'Albania', 'isCountry': True, 'countryCode': 'AL'}]

Toda la información está disponible en la [documentación de la API](https://developer.clashroyale.com/#/documentation), a la que se puede acceder iniciando sesión.

---

<a id="section3"></a>
## <font color="#00586D"> 3. Metodología</font>

En primer lugar, debemos definir claramente el problema que queremos resolver. Debemos tener en cuenta que Clash Royale cuenta con varios modos de juego en los que las reglas son diferentes, por lo que no es adecuado desarrollar un modelo que abarque todo tipo de partidas. En este proyecto, nos enfocaremos en partidas 1vs1 del modo *Camino de Trofeos*. De este modo, podremos predecir partidas entre jugadores sin aplicar restricciones adicionales de otros modos de juego más recientes y menos comunes. Hay que tener en cuenta que también se podrán simular partidas amistosas entre jugadores dispares, pero no debemos olvidar que aprenderemos de partidas entre jugadores con copas muy similares fruto del propio emparejamiento del juego.

Como futura mejora se propone desarrollar modelos para modos de juego competitivos como *Camino de Leyendas*, lo cual es una tarea más compleja al igualarse el nivel de las cartas.

Una vez sabemos qué tipo de datos necesitamos, debemos encontrar la forma de extraer la información de las partidas. Como hemos visto, la única manera es mediante el registro de batalla de los jugadores.

Después de hacer diferentes consideraciones al respecto, se ha decidido proceder de la siguiente manera:

1. Obtener tags (únicos) de clanes aleatorios que tengan una cantidad considerable de miembros (`/clans`).
2. Obtener tags de jugadores a través de los miembros de los clanes anteriores (`/clans{clanTag}/members`).
3. Obtener la última batalla de los jugadores anteriores (`players/{playerTag}/battlelog`).
    * Es posible que haya jugadores sin partidas recientes.
    * Sólo se tendrán en cuenta las partidas de *Ladder* correspondientes al modo de juego *Camino de Trofeos*.
    * En la siguiente etapa del proyecto se limpiarán y transformarán los datos para crear el *dataset* final.

Además, la información de las cartas se complementará con propiedades adicionales extraidas de otras fuentes (tipos, counters...)

Se ha elegido esta metodología principalmente para obtener una cantidad de datos manejable y que a la misma vez nos permita crear modelos que generalicen. Hubiera sido posible obtener todo el registro de batalla de los jugadores, pero la cantidad de información crecería enormemente. Si tenemos en cuenta que el conjunto de datos con el que vamos a trabajar tampoco puede ser excesivamente grande, se ha considerado que es preferible quedarnos solamente con la última partida de cada jugador (y solamente si es de *Ladder*). Dispuestos a tener un número de ejemplos similar en nuestro conjunto de datos, una única partida por jugador podría proporcionar una mayor variedad, ya que un mismo jugador no suele cambiar frecuentemente de mazo y tiene una forma determinada de jugar. Por lo tanto, la estrategia elegida tiene como objetivo adquirir información de partidas de modo que esta sea variada y apta para los recursos disponibles.

#### <font color="#00586D"> Consideraciones sobre integridad temporal</font>

Con el objetivo de enriquecer el conjunto de datos, se ha evaluado la posibilidad de complementar los registros de partidas con información adicional de los perfiles de los jugadores que aporte valor (nivel de experiencia, tasa de victorias, récord de trofeos o maestría de cartas). Esta información es accesible a través de la API oficial de Clash Royale como hemos visto anteriormente, pero presenta una limitación importante: la API solo devuelve el estado actual del perfil del jugador, sin conservar versiones anteriores que permitan reconstruir su estado exacto en el momento de cada partida.

Este desfase temporal introduce un riesgo de fuga de datos temporal. Dado que muchas de estas variables del perfil pueden haber cambiado desde el momento en que se jugó la partida (por ejemplo, un jugador podría haber ganado más puntos de experiencia o podría haber alcanzado un nuevo récord de trofeos), estaríamos entrenando el modelo con información generada después del evento que se quiere predecir. Este tipo de sesgo, también conocido como *look-ahead bias*, puede llevar a que el modelo aprenda patrones irreales.

Aunque en un escenario de predicción real el perfil del jugador sí estará disponible antes de que ocurra una nueva partida (que es lo que queremos predecir), el problema radica en el entrenamiento. Al usar partidas antiguas complementadas con perfiles actuales, se estaría asumiendo erróneamente que esa información es contemporánea al evento. Por esta razón, se ha optado por una estrategia conservadora: utilizar exclusivamente los datos disponibles en el propio registro de la partida.

Si realmente queremos explorar esta posibilidad en el futuro para mejorar el rendimiento del modelo, podríamos recurrir a las siguientes opciones:
* Asumir el margen de error temporal y complementar con los datos del perfil solo para partidas muy recientes.
* Implementar un sistema que recopile partidas en tiempo real, capturando también la información de los jugadores en ese momento.

---

<a id="section4"></a>
## <font color="#00586D"> 4. Adquisición y almacenamiento</font>

A lo largo de esta sección se va a llevar a cabo el proceso de extracción y almacenamiento de datos utilizando la API de Clash Royale e información de otras fuentes.

<a id="section4_1"></a>
### <font color="#00586D"> 4.1. Cartas</font>

Para las cartas, lo primero que haremos será guardar la información devuelta por la API:

In [19]:
with open("../data/raw/cards.json", "w", encoding="utf-8") as file:
    json.dump(cards["items"], file, ensure_ascii=False, indent=4)

Lo mismo con las tropas de las torres de coronas:

In [20]:
with open("../data/raw/support_cards.json", "w", encoding="utf-8") as file:
    json.dump(cards["supportItems"], file, ensure_ascii=False, indent=4)

Vamos a complementar la información de las cartas con otros datos relevantes extraídos de otras fuentes como [RoyaleAPI](https://royaleapi.com/) y [DeckShop](https://www.deckshop.pro/).

#### <font color="#00586D">Win Conditions</font>

Cartas que suele usarse para hacer daño directo a las torres enemigas y ganar la partida. La mayoría coinciden con las tropas que tienen los edificios como objetivo.

In [21]:
win_conditions = [
    "Lava Hound",
    "Giant",
    "Electro Giant",
    "Goblin Giant",
    "Golem",
    "Three Musketeers",
    "Battle Ram",
    "Balloon",
    "Goblin Drill",
    "Mortar",
    "Wall Breakers",
    "Skeleton Barrel",
    "Miner",
    "Sparky",
    "Royal Giant",
    "Graveyard",
    "Royal Hogs",
    "Ram Rider",
    "X-Bow",
    "Elixir Golem",
    "Hog Rider",
    "Goblin Barrel"
]

#### <font color="#00586D">Cuerpo a cuerpo</font>

Carta que realizan ataques cuerpo a cuerpo (a muy corta distancia).

In [22]:
melee = [
    "Skeletons",
    "Goblins",
    "Berserker",
    "Goblin Gang",
    "Barbarians",
    "Rascals",
    "Elixir Golem",
    "Mini P.E.K.K.A",
    "Goblin Cage",
    "Barbarian Barrel",
    "Goblin Barrel",
    "Skeleton Army",
    "Giant Skeleton",
    "Bandit",
    "Lumberjack",
    "Graveyard",
    "Bats",
    "Knight",
    "Elite Barbarians",
    "Valkyrie",
    "Dark Prince",
    "Rune Giant",
    "Goblin Giant",
    "P.E.K.K.A",
    "Electro Giant",
    "Miner",
    "Royal Ghost",
    "Fisherman",
    "Goblin Machine",
    "Mega Knight",
    "Golden Knight",
    "Skeleton King",
    "Goblinstein",
    "Monk",
    "Minions",
    "Royal Delivery",
    "Minion Horde",
    "Royal Recruits",
    "Mega Minion",
    "Battle Healer",
    "Guards",
    "Prince",
    "Phoenix",
    "Night Witch",
    "Mighty Miner",
    "Boss Bandit"
]

#### <font color="#00586D">A distancia</font>

Cartas que realizan ataques a distancia.

In [23]:
ranged = [
    "Spear Goblins",
    "Bomber",
    "Archers",
    "Cannon",
    "Goblin Gang",
    "Firecracker",
    "Skeleton Dragons",
    "Rascals",
    "Dart Goblin",
    "Musketeer",
    "Flying Machine",
    "Zappies",
    "Goblin Demolisher",
    "Goblin Hut",
    "Wizard",
    "Three Musketeers",
    "Baby Dragon",
    "Hunter",
    "Witch",
    "Electro Dragon",
    "Bowler",
    "Executioner",
    "Cannon Cart",
    "Goblin Giant",
    "X-Bow",
    "Princess",
    "Ice Wizard",
    "Fisherman",
    "Electro Wizard",
    "Inferno Dragon",
    "Magic Archer",
    "Mother Witch",
    "Ram Rider",
    "Sparky",
    "Little Prince",
    "Archer Queen",
    "Goblinstein"
]

#### <font color="#00586D">Tropas aéreas</font>

Cartas que son unidades aéreas, es decir, que pueden volar y solo pueden ser atacadas por cartas con capacidad antiaérea.

In [24]:
air = [
    "Bats",
    "Minions",
    "Skeleton Barrel",
    "Skeleton Dragons",
    "Minion Horde",
    "Mega Minion",
    "Flying Machine",
    "Baby Dragon",
    "Balloon",
    "Electro Dragon",
    "Inferno Dragon",
    "Lava Hound",
    "Phoenix"
]

#### <font color="#00586D">Daño aéreo</font>

Cartas con capacidad antiaérea, es decir, que pueden hacer daño a unidades aéreas.

In [25]:
anti_air = [
    "Electro Spirit",
    "Fire Spirit",
    "Ice Spirit",
    "Spear Goblins",
    "Bats", 
    "Zap",
    "Giant Snowball",
    "Archers",
    "Arrows",
    "Minions",
    "Goblin Gang",
    "Firecracker",
    "Royal Delivery",
    "Skeleton Dragons",
    "Tesla",
    "Minion Horde",
    "Rascals",
    "Ice Golem",
    "Mega Minion",
    "Dart Goblin",
    "Fireball",
    "Musketeer",
    "Flying Machine",
    "Furnace",
    "Zappies",
    "Goblin Hut",
    "Inferno Tower",
    "Wizard",
    "Rocket",
    "Three Musketeers",
    "Goblin Curse",
    "Rage",
    "Tornado",
    "Void",
    "Baby Dragon",
    "Freeze",
    "Poison",
    "Hunter",
    "Witch",
    "Electro Dragon",
    "Executioner",
    "Lightning",
    "Goblin Giant",
    "Electro Giant",
    "Princess",
    "Ice Wizard",
    "Electro Wizard",
    "Inferno Dragon",
    "Phoenix",
    "Magic Archer",
    "Night Witch",
    "Mother Witch",
    "Ram Rider",
    "Little Prince",
    "Archer Queen",
    "Goblinstein"
]

#### <font color="#00586D">Daño directo</font>

Cartas que pueden hacer daño a las torres sin necesidad de desplegar o invocar. Todas son hechizos, pero no todos los hechizos son cartas de daño directo.

In [26]:
direct_damage = [
    "Zap",
    "Giant Snowball",
    "Arrows",
    "Royal Delivery",
    "Earthquake",
    "Fireball",
    "Rocket",
    "Barbarian Barrel",
    "Goblin Curse",
    "Rage",
    "Tornado",
    "Void",
    "Freeze",
    "Poison",
    "Lightning",
    "The Log"
]

#### <font color="#00586D">Daño de salpicadura</font>
Cartas que causan daño en área, afectando a varios objetivos cercanos al mismo tiempo.

In [27]:
splash_damage = [
    "Electro Spirit",
    "Fire Spirit",
    "Ice Spirit",
    "Bomber",
    "Zap",
    "Giant Snowball",
    "Arrows",
    "Firecracker",
    "Royal Delivery",
    "Skeleton Dragons",
    "Mortar",
    "Ice Golem",
    "Earthquake",
    "Fireball",
    "Valkyrie",
    "Bomb Tower",
    "Furnace",
    "Goblin Demolisher",
    "Wizard",
    "Rocket",
    "Barbarian Barrel",
    "Tornado",
    "Baby Dragon",
    "Dark Prince",
    "Freeze",
    "Poison",
    "Hunter",
    "Electro Dragon",
    "Bowler",
    "Executioner",
    "Electro Giant",
    "The Log",
    "Princess",
    "Ice Wizard",
    "Royal Ghost",
    "Magic Archer",
    "Goblin Machine",
    "Sparky",
    "Mega Knight",
    "Skeleton King"
]

#### <font color="#00586D">Reseteo</font>
Cartas con la capacidad de reiniciar ataques cargados o habilidades enemigas.

In [28]:
reset_attack = [
    "Electro Spirit",
    "Ice Spirit",
    "Zap",
    "Zappies",
    "Freeze",
    "Electro Dragon",
    "Lightning",
    "Electro Giant",
    "Electro Wizard",
    "Goblinstein"
]

Como todos estos atributos son *booleanos*, podemos comprobar si están en las listas correspondientes para asignarles el valor.

In [29]:
card_properties = [
    {
        "id": card["id"],
        "name": card["name"],
        "winCondition": card["name"] in win_conditions,
        "melee": card["name"] in melee,
        "ranged": card["name"] in ranged,
        "air": card["name"] in air,
        "antiAir": card["name"] in anti_air,
        "directDamage": card["name"] in direct_damage,
        "splashDamage": card["name"] in splash_damage,
        "resetAttack": card["name"] in reset_attack
    }
    for card in cards["items"]
]

También vamos a incluir el tipo de la carta (`troop`, `building` o `spell`). Es posible distinguir los tipos a través del identificador devuelto por la API (a excepción, por alguna razón, del *Espíritu sanador*).

In [30]:
for card in card_properties:
    if str(card["id"]).startswith("27"):
        card["type"] = "building"
    elif str(card["id"]).startswith("28") and card["name"] != "Heal Spirit":
        card["type"] = "spell"
    else:
        card["type"] = "troop"

El resultado es el siguiente:

In [31]:
print(json.dumps(card_properties, indent=4, ensure_ascii=False))

[
    {
        "id": 26000000,
        "name": "Knight",
        "winCondition": false,
        "melee": true,
        "ranged": false,
        "air": false,
        "antiAir": false,
        "directDamage": false,
        "splashDamage": false,
        "resetAttack": false,
        "type": "troop"
    },
    {
        "id": 26000001,
        "name": "Archers",
        "winCondition": false,
        "melee": false,
        "ranged": true,
        "air": false,
        "antiAir": true,
        "directDamage": false,
        "splashDamage": false,
        "resetAttack": false,
        "type": "troop"
    },
    {
        "id": 26000002,
        "name": "Goblins",
        "winCondition": false,
        "melee": true,
        "ranged": false,
        "air": false,
        "antiAir": false,
        "directDamage": false,
        "splashDamage": false,
        "resetAttack": false,
        "type": "troop"
    },
    {
        "id": 26000003,
        "name": "Giant",
        "winCondition": 

Una vez que los registros son correctos, vamos a guardar la información.

In [32]:
with open("../data/raw/card_properties.json", "w", encoding="utf-8") as file:
    json.dump(card_properties, file, ensure_ascii=False, indent=4)

#### <font color="#00586D">Counters</font>

Además de la información de las cartas y las propiedades que acabamos de guardar, un factor que puede resultar determinante en el desenlace de una partida son los *counters*. Una carta es *counter* de otra cuando sus características son extremadamente efectivas para neutralizarla. Estos datos no son accesibles a través de la API y por lo que ha sido necesario investigar para acabar definiéndolos manualmente, lo que ha supuesto un gran trabajo. Para poder almacenar la información, se ha utilizado un diccionario donde la clave es el nombre de la carta y el valor es un diccionario con un único atributo `counters`. Este puede leerse fácilemente con *Pandas* utilizando `pd.read_json()` con `orient="index"`.

La información de los *counters* se ha extraido de [DeckShop](https://www.deckshop.pro/). Es importante tener en cuenta que realmente cada carta tiene una gran cantidad de cartas capaces de contrarrestarlas, pero nos hemos enfocado en aquellas que lo hacen de manera realmente efectiva en ataque o en defensa. También hay cartas que no tienen *counters* y por lo tanto no aparecen, como la mayoría de los hechizos.

In [33]:
card_counters = {
    "Knight": {
        "counters": [
            "Knight",
            "Royal Delivery",
            "Ice Golem",
            "Dart Goblin",
            "Mini P.E.K.K.A",
            "Valkyrie",
            "Battle Healer",
            "Inferno Tower",
            "Guards",
            "Skeleton Army",
            "Dark Prince",
            "Hunter",
            "Prince",
            "Electro Dragon",
            "Bowler",
            "Executioner",
            "Cannon Cart",
            "Ice Wizard",
            "Bandit",
            "Fisherman",
            "Electro Wizard",
            "Lumberjack",
            "Night Witch",
            "Sparky"
        ]
    },
    "Archers": {
        "counters": [
            "Arrows",
            "Firecracker",
            "Royal Delivery",
            "Skeleton Dragons",
            "Royal Recruits",
            "Fireball",
            "Bomb Tower",
            "Barbarian Hut",
            "Skeleton Army",
            "Poison",
            "Bowler",
            "X-Bow",
            "Sparky",
            "Monk"
        ]
    },
    "Goblins": {
        "counters": [
            "Fire Spirit",
            "Arrows",
            "Skeleton Dragons",
            "Earthquake",
            "Fireball",
            "Valkyrie",
            "Bomb Tower",
            "Goblin Demolisher",
            "Wizard",
            "Barbarian Barrel",
            "Skeleton Army",
            "Poison",
            "Bowler",
            "X-Bow",
            "The Log",
            "Royal Ghost",
            "Sparky",
            "Mega Knight",
            "Bomber",
            "Electro Giant"
        ]
    },
    "Giant": {
        "counters": [
            "Cannon",
            "Tesla",
            "Barbarians",
            "Minion Horde",
            "Elite Barbarians",
            "Royal Recruits",
            "Mini P.E.K.K.A",
            "Goblin Cage",
            "Bomb Tower",
            "Goblin Hut",
            "Inferno Tower",
            "Barbarian Hut",
            "Three Musketeers",
            "Skeleton Army",
            "Hunter",
            "Witch",
            "Prince",
            "P.E.K.K.A",
            "Inferno Dragon",
            "Lumberjack",
            "Night Witch",
            "Sparky"
        ]
    },
    "P.E.K.K.A": {
        "counters": [
            "Royal Recruits",
            "Inferno Tower",
            "Witch",
            "P.E.K.K.A"
        ]
    },
    "Minions": {
        "counters": [
            "Ice Spirit",
            "Arrows",
            "Firecracker",
            "Royal Delivery",
            "Fireball",
            "Wizard",
            "Baby Dragon",
            "Poison",
            "Executioner",
            "Electro Wizard"
        ]
    },
    "Balloon": {
        "counters": [
            "Bats",
            "Giant Snowball",
            "Minions",
            "Skeleton Dragons",
            "Tesla",
            "Minion Horde",
            "Mega Minion",
            "Dart Goblin",
            "Musketeer",
            "Furnace",
            "Inferno Tower",
            "Rocket",
            "Three Musketeers",
            "Tornado",
            "Hunter",
            "Lightning",
            "Inferno Dragon",
            "Electro Wizard",
            "Ram Rider"
        ]
    },
    "Witch": {
        "counters": [
            "Royal Delivery",
            "Valkyrie",
            "Rocket",
            "Bowler",
            "Executioner",
            "Lightning",
            "Mega Knight"
        ]
    },
    "Barbarians": {
        "counters": [
            "Bomber",
            "Mortar",
            "Royal Recruits",
            "Fireball",
            "Valkyrie",
            "Bomb Tower",
            "Furnace",
            "Goblin Demolisher",
            "Wizard",
            "Poison",
            "Rocket",
            "Bowler",
            "Executioner",
            "Sparky",
            "Mega Knight",
            "Goblinstein"
        ]
    },
    "Golem": {
        "counters": [
            "Cannon",
            "Tesla",
            "Barbarians",
            "Minion Horde",
            "Mini P.E.K.K.A",
            "Zappies",
            "Inferno Tower",
            "Barbarian Hut",
            "Three Musketeers",
            "Hunter",
            "Witch",
            "P.E.K.K.A",
            "Inferno Dragon",
            "Sparky",
            "Mighty Miner"
        ]
    },
    "Skeletons": {
        "counters": [
            "Electro Spirit",
            "Zap",
            "Giant Snowball",
            "Arrows",
            "Firecracker",
            "Earthquake",
            "Goblin Demolisher",
            "Wizard",
            "Goblin Curse",
            "Tornado",
            "Dark Prince",
            "Freeze",
            "Poison",
            "Electro Dragon",
            "Bowler",
            "Executioner",
            "X-Bow",
            "Electro Giant",
            "The Log",
            "Princess",
            "Ice Wizard",
            "Mother Witch",
            "Ram Rider"
        ]
    },
    "Valkyrie": {
        "counters": [
            "Minion Horde",
            "Inferno Tower",
            "Sparky"
        ]
    },
    "Skeleton Army": {
        "counters": [
            "Bomber",
            "Zap",
            "Giant Snowball",
            "Arrows",
            "Royal Delivery",
            "Skeleton Dragons",
            "Ice Golem",
            "Earthquake",
            "Valkyrie",
            "Goblin Demolisher",
            "Wizard",
            "Barbarian Barrel",
            "Goblin Curse",
            "Freeze",
            "Poison",
            "Witch",
            "Bowler",
            "Executioner",
            "The Log",
            "Electro Wizard",
            "Mega Knight",
            "Skeleton King",
            "Firecracker",
            "Tornado",
            "Electro Giant",
            "Mother Witch"
        ]
    },
    "Bomber": {
        "counters": [
            "Knight",
            "Royal Delivery",
            "Fireball",
            "Valkyrie",
            "Barbarian Barrel",
            "Void",
            "Poison",
            "Dark Prince",
            "Prince",
            "Miner",
            "Sparky",
            "Mega Knight",
            "Monk"
        ]
    },
    "Musketeer": {
        "counters": [
            "Firecracker",
            "Elite Barbarians",
            "Rocket",
            "Skeleton Army",
            "Void",
            "Poison",
            "Lightning",
            "Miner",
            "Lumberjack",
            "Monk",
            "Sparky",
            "Mega Knight"
        ]
    },
    "Baby Dragon": {
        "counters": [
            "Tesla",
            "Rascals",
            "Mega Minion",
            "Musketeer",
            "Inferno Tower",
            "Rocket",
            "Baby Dragon",
            "Inferno Dragon"
        ]
    },
    "Prince": {
        "counters": [
            "Goblin Gang",
            "Royal Delivery",
            "Barbarians",
            "Royal Recruits",
            "Tombstone",
            "Zappies",
            "Guards",
            "Skeleton Army",
            "Giant Skeleton",
            "P.E.K.K.A",
            "Bandit",
            "Lumberjack",
            "Mega Knight"
        ]
    },
    "Wizard": {
        "counters": [
            "Knight",
            "Royal Delivery",
            "Elite Barbarians",
            "Mini P.E.K.K.A",
            "Valkyrie",
            "Rocket",
            "Barbarian Barrel",
            "Tornado",
            "Void",
            "Poison",
            "Dark Prince",
            "Hunter",
            "Prince",
            "Giant Skeleton",
            "Lightning",
            "P.E.K.K.A",
            "Miner",
            "Mega Knight",
            "Monk"
        ]
    },
    "Mini P.E.K.K.A": {
        "counters": [
            "Goblin Gang",
            "Royal Delivery",
            "Rascals",
            "Royal Recruits",
            "Mini P.E.K.K.A",
            "Rocket",
            "Lightning",
            "Guards",
            "Skeleton Army",
            "Dark Prince",
            "Prince",
            "P.E.K.K.A",
            "Sparky"
        ]
    },
    "Spear Goblins": {
        "counters": [
            "Fire Spirit",
            "Zap",
            "Giant Snowball",
            "Arrows",
            "Firecracker",
            "Royal Delivery",
            "Skeleton Dragons",
            "Ice Golem",
            "Earthquake",
            "Fireball",
            "Valkyrie",
            "Bomb Tower",
            "Goblin Demolisher",
            "Wizard",
            "Barbarian Barrel",
            "Skeleton Army",
            "Tornado",
            "Baby Dragon",
            "Freeze",
            "Poison",
            "Bowler",
            "Executioner",
            "X-Bow",
            "The Log",
            "Princess",
            "Royal Ghost",
            "Electro Wizard",
            "Mother Witch",
            "Sparky",
            "Mega Knight"
        ]
    },
    "Giant Skeleton": {
        "counters": [
            "Barbarians",
            "Royal Recruits",
            "Inferno Tower",
            "Barbarian Hut",
            "Giant Skeleton",
            "P.E.K.K.A"
        ]
    },
    "Hog Rider": {
        "counters": [
            "Cannon",
            "Goblin Gang",
            "Mortar",
            "Tesla",
            "Barbarians",
            "Minion Horde",
            "Mini P.E.K.K.A",
            "Goblin Cage",
            "Bomb Tower",
            "Furnace",
            "Goblin Hut",
            "Inferno Tower",
            "Rocket",
            "Barbarian Hut",
            "Three Musketeers",
            "Skeleton Army",
            "Tornado",
            "Hunter",
            "Witch",
            "Prince",
            "Bowler",
            "P.E.K.K.A",
            "Fisherman",
            "Lumberjack",
            "Ram Rider",
            "Sparky",
            "Mega Knight"
        ]
    },
    "Minion Horde": {
        "counters": [
            "Fire Spirit",
            "Arrows",
            "Firecracker",
            "Royal Delivery",
            "Skeleton Dragons",
            "Fireball",
            "Furnace",
            "Wizard",
            "Rocket",
            "Poison",
            "Executioner",
            "Electro Giant",
            "Mother Witch",
            "Goblinstein"
        ]
    },
    "Ice Wizard": {
        "counters": [
            "Fireball",
            "Void",
            "Lightning"
        ]
    },
    "Royal Giant": {
        "counters": [
            "Barbarians",
            "Minion Horde",
            "Elite Barbarians",
            "Mini P.E.K.K.A",
            "Bomb Tower",
            "Inferno Tower",
            "Barbarian Hut",
            "Skeleton Army",
            "Hunter",
            "P.E.K.K.A",
            "Inferno Dragon",
            "Lumberjack",
            "Sparky"
        ]
    },
    "Guards": {
        "counters": [
            "Fire Spirit",
            "Bomber",
            "Firecracker",
            "Royal Delivery",
            "Skeleton Dragons",
            "Royal Recruits",
            "Valkyrie",
            "Furnace",
            "Skeleton Army",
            "Baby Dragon",
            "Poison",
            "Witch",
            "Bowler",
            "Mega Knight"
        ]
    },
    "Princess": {
        "counters": [
            "Arrows",
            "Firecracker",
            "Mortar",
            "Royal Delivery",
            "Fireball",
            "Valkyrie",
            "Barbarian Barrel",
            "Void",
            "Poison",
            "Hunter",
            "X-Bow",
            "The Log",
            "Miner",
            "Electro Wizard"
        ]
    },
    "Dark Prince": {
        "counters": [
            "Knight",
            "Rascals",
            "Elite Barbarians",
            "Royal Recruits",
            "Tombstone",
            "Mini P.E.K.K.A",
            "Valkyrie",
            "Dark Prince",
            "P.E.K.K.A",
            "Bandit"
        ]
    },
    "Three Musketeers": {
        "counters": [
            "Lightning",
            "Rocket",
            "Fireball",
            "Monk"
        ]
    },
    "Lava Hound": {
        "counters": [
            "Minions",
            "Arrows",
            "Skeleton Dragons",
            "Tesla",
            "Minion Horde",
            "Mega Minion",
            "Musketeer",
            "Inferno Tower",
            "Three Musketeers",
            "Hunter",
            "Executioner",
            "Inferno Dragon"
        ]
    },
    "Ice Spirit": {
        "counters": [
            "Arrows",
            "Royal Delivery",
            "Skeleton Dragons",
            "Fireball",
            "Bomb Tower",
            "Barbarian Barrel",
            "Skeleton Army",
            "X-Bow",
            "The Log",
            "Sparky",
            "Mega Knight"
        ]
    },
    "Fire Spirit": {
        "counters": [
            "Skeletons",
            "Zap",
            "Giant Snowball",
            "Arrows",
            "Knight",
            "Royal Delivery",
            "Skeleton Dragons",
            "Ice Golem",
            "Tombstone",
            "Fireball",
            "Valkyrie",
            "Battle Healer",
            "Inferno Tower",
            "Barbarian Barrel",
            "Skeleton Army",
            "Tornado",
            "Baby Dragon",
            "Dark Prince",
            "Freeze",
            "Prince",
            "Bowler",
            "Cannon Cart",
            "Giant Skeleton",
            "The Log",
            "Bandit",
            "Electro Wizard",
            "Sparky",
            "Mega Knight"
        ]
    },
    "Miner": {
        "counters": [
            "Knight",
            "Goblin Gang",
            "Elite Barbarians",
            "Guards",
            "Skeleton Army",
            "Tornado"
        ]
    },
    "Sparky": {
        "counters": [
            "Zap",
            "Royal Delivery",
            "Minion Horde",
            "Royal Recruits",
            "Goblin Cage",
            "Rocket",
            "Void",
            "Freeze",
            "Electro Dragon",
            "Giant Skeleton",
            "Lightning",
            "P.E.K.K.A",
            "Electro Wizard",
            "Mega Knight",
            "Goblinstein"
        ]
    },
    "Bowler": {
        "counters": [
            "Inferno Tower",
            "P.E.K.K.A",
            "Inferno Dragon",
            "Sparky",
            "Mega Knight"
        ]
    },
    "Lumberjack": {
        "counters": [
            "Royal Delivery",
            "Minion Horde",
            "Royal Recruits",
            "Mini P.E.K.K.A",
            "Barbarian Hut",
            "Guards",
            "Skeleton Army",
            "Bowler",
            "Lightning",
            "P.E.K.K.A",
            "Mega Knight"
        ]
    },
    "Battle Ram": {
        "counters": [
            "Cannon",
            "Mortar",
            "Tesla",
            "Barbarians",
            "Minion Horde",
            "Royal Recruits",
            "Tombstone",
            "Mini P.E.K.K.A",
            "Goblin Cage",
            "Bomb Tower",
            "Furnace",
            "Goblin Hut",
            "Inferno Tower",
            "Barbarian Hut",
            "Skeleton Army",
            "Hunter",
            "Goblin Drill",
            "Prince",
            "Bowler",
            "P.E.K.K.A",
            "Lumberjack",
            "Night Witch",
            "Sparky",
            "Mega Knight"
        ]
    },
    "Inferno Dragon": {
        "counters": [
            "Spear Goblins",
            "Zap",
            "Bats",
            "Minion Horde",
            "Zappies",
            "Rocket",
            "Electro Dragon",
            "Lightning",
            "Electro Wizard",
            "Goblinstein"
        ]
    },
    "Ice Golem": {
        "counters": [
            "Inferno Tower",
            "Sparky",
            "Mega Knight"
        ]
    },
    "Mega Minion": {
        "counters": [
            "Spear Goblins",
            "Minion Horde",
            "Firecracker",
            "Void",
            "Wizard",
            "Executioner"
        ]
    },
    "Dart Goblin": {
        "counters": [
            "Arrows",
            "Knight",
            "Royal Delivery",
            "Fireball",
            "Valkyrie",
            "Barbarian Barrel",
            "Void",
            "Poison",
            "X-Bow",
            "The Log",
            "Royal Ghost",
            "Monk"
        ]
    },
    "Goblin Gang": {
        "counters": [
            "Arrows",
            "Royal Delivery",
            "Earthquake",
            "Fireball",
            "Valkyrie",
            "Bomb Tower",
            "Furnace",
            "Goblin Demolisher",
            "Barbarian Barrel",
            "Poison",
            "Bowler",
            "Executioner",
            "The Log",
            "Sparky",
            "Mega Knight"
        ]
    },
    "Electro Wizard": {
        "counters": [
            "Royal Delivery",
            "Barbarians",
            "Mini P.E.K.K.A",
            "Valkyrie",
            "Rocket",
            "Void",
            "Poison",
            "Lightning",
            "Mega Knight"
        ]
    },
    "Elite Barbarians": {
        "counters": [
            "Royal Delivery",
            "Barbarians",
            "Rascals",
            "Elite Barbarians",
            "Royal Recruits",
            "Mini P.E.K.K.A",
            "Rocket",
            "Skeleton Army",
            "Dark Prince",
            "Prince",
            "Bowler",
            "Cannon Cart",
            "Giant Skeleton",
            "Lightning",
            "P.E.K.K.A",
            "Mega Knight",
            "Sparky"
        ]
    },
    "Hunter": {
        "counters": [
            "Royal Delivery",
            "Barbarians",
            "Royal Recruits",
            "Valkyrie",
            "Rocket",
            "Void",
            "Prince",
            "Cannon Cart",
            "Lightning",
            "Mega Knight"
        ]
    },
    "Executioner": {
        "counters": [
            "Mini P.E.K.K.A",
            "Valkyrie",
            "Rocket",
            "Void",
            "Giant Skeleton",
            "P.E.K.K.ALumberjack",
            "Mega Knight"
        ]
    },
    "Bandit": {
        "counters": [
            "Knight",
            "Goblin Gang",
            "Barbarians",
            "Rascals",
            "Elite Barbarians",
            "Royal Recruits",
            "Mini P.E.K.K.A",
            "Valkyrie",
            "Barbarian Hut",
            "Guards",
            "Skeleton Army",
            "Lumberjack",
            "Mega Knight"
        ]
    },
    "Royal Recruits": {
        "counters": [
            "Royal Recruits",
            "Wizard"
        ]
    },
    "Night Witch": {
        "counters": [
            "Elite Barbarians",
            "Poison",
            "Lightning"
        ]
    },
    "Bats": {
        "counters": [
            "Electro Spirit",
            "Ice Spirit",
            "Zap",
            "Giant Snowball",
            "Spear Goblins",
            "Arrows",
            "Skeleton Barrel",
            "Firecracker",
            "Royal Delivery",
            "Skeleton Dragons",
            "Ice Golem",
            "Fireball",
            "Wizard",
            "Goblin Curse",
            "Tornado",
            "Baby Dragon",
            "Freeze",
            "Poison",
            "Executioner",
            "Electro Giant",
            "Princess",
            "Ice Wizard",
            "Electro Wizard",
            "Mother Witch"
        ]
    },
    "Royal Ghost": {
        "counters": [
            "Knight",
            "Royal Delivery",
            "Ice Golem",
            "Mini P.E.K.K.A",
            "Valkyrie",
            "Battle Healer",
            "Barbarian Hut",
            "Guards",
            "Skeleton Army",
            "Dark Prince",
            "Lightning",
            "Electro Wizard",
            "Lumberjack"
        ]
    },
    "Ram Rider": {
        "counters": [
            "Barbarians",
            "Minion Horde",
            "Bomb Tower",
            "Inferno Tower",
            "Rocket",
            "Barbarian Hut",
            "Skeleton Army",
            "Tornado",
            "Monk"
        ]
    },
    "Zappies": {
        "counters": [
            "Bomber",
            "Royal Delivery",
            "Firecracker",
            "Earthquake",
            "Fireball",
            "Valkyrie",
            "Wizard",
            "Poison",
            "Bowler",
            "Mega Knight"
        ]
    },
    "Rascals": {
        "counters": [
            "Arrows",
            "Royal Delivery",
            "Barbarians",
            "Elite Barbarians",
            "Royal Recruits",
            "Fireball",
            "Valkyrie",
            "Rocket",
            "Poison",
            "The Log",
            "Mega Knight"
        ]
    },
    "Cannon Cart": {
        "counters": [
            "Goblin Gang",
            "Royal Recruits",
            "Mini P.E.K.K.A",
            "Valkyrie",
            "Rocket",
            "Guards",
            "Skeleton Army",
            "Dark Prince",
            "Hunter",
            "Prince",
            "Bowler",
            "Giant Skeleton",
            "Lightning",
            "P.E.K.K.A",
            "Lumberjack",
            "Mega Knight"
        ]
    },
    "Mega Knight": {
        "counters": [
            "Mini P.E.K.K.A",
            "Inferno Tower",
            "Giant Skeleton",
            "P.E.K.K.A",
            "Mega Knight",
            "Mighty Miner"
        ]
    },
    "Skeleton Barrel": {
        "counters": [
            "Zap",
            "Giant Snowball",
            "Arrows",
            "Royal Delivery",
            "Mortar",
            "Earthquake",
            "Goblin Cage",
            "Valkyrie",
            "Bomb Tower",
            "Furnace",
            "Wizard",
            "Barbarian Barrel",
            "Tornado",
            "Baby Dragon",
            "Freeze",
            "Poison",
            "Hunter",
            "Goblin Drill",
            "Witch",
            "Executioner",
            "The Log"
        ]
    },
    "Flying Machine": {
        "counters": [
            "Skeleton Dragons",
            "Mega Minion",
            "Fireball",
            "Wizard",
            "Void",
            "Poison",
            "Lightning",
            "Electro Wizard",
            "Monk"
        ]
    },
    "Wall Breakers": {
        "counters": [
            "Fire Spirit",
            "Ice Spirit",
            "Arrows",
            "Cannon",
            "Royal Delivery",
            "Mortar",
            "Tesla",
            "Tombstone",
            "Fireball",
            "Goblin Cage",
            "Valkyrie",
            "Bomb Tower",
            "Mega Knight"
        ]
    },
    "Royal Hogs": {
        "counters": [
            "Fire Spirit",
            "Bomber",
            "Fireball",
            "Valkyrie",
            "Bomb Tower",
            "Wizard",
            "Rocket",
            "Barbarian Hut",
            "Three Musketeers",
            "Skeleton Army",
            "Bowler",
            "Executioner",
            "Sparky",
            "Mega Knight"
        ]
    },
    "Goblin Giant": {
        "counters": [
            "Barbarians",
            "Elite Barbarians",
            "Royal Recruits",
            "Mini P.E.K.K.A",
            "Inferno Tower",
            "Barbarian Hut",
            "Skeleton Army",
            "P.E.K.K.A",
            "Inferno Dragon",
            "Sparky"
        ]
    },
    "Fisherman": {
        "counters": [
            "Royal Recruits",
            "Barbarian Hut",
            "Electro Wizard",
            "Mega Knight"
        ]
    },
    "Magic Archer": {
        "counters": [
            "Royal Delivery",
            "Fireball",
            "Rocket",
            "Void",
            "Poison",
            "Lightning",
            "Miner",
            "Bandit",
            "Electro Wizard"
        ]
    },
    "Electro Dragon": {
        "counters": [
            "Royal Delivery",
            "Musketeer",
            "Flying Machine",
            "Wizard",
            "Rocket",
            "Lightning"
        ]
    },
    "Firecracker": {
        "counters": [
            "Arrows",
            "Firecracker",
            "Mortar",
            "Cannon",
            "Tesla",
            "Mega Minion",
            "Fireball",
            "Musketeer",
            "Barbarian Barrel",
            "Void",
            "Poison",
            "X-Bow",
            "Bandit",
            "Electro Wizard",
            "Mega Knight",
            "Golden Knight",
            "Monk"
        ]
    },
    "Mighty Miner": {
        "counters": [
            "Zap",
            "Goblin Gang",
            "Tesla",
            "Barbarians",
            "Rascals",
            "Elite Barbarians",
            "Royal Recruits",
            "Tombstone",
            "Zappies",
            "Goblin Hut",
            "Inferno Tower",
            "Guards",
            "Skeleton Army",
            "Witch",
            "Electro DragonElectro Giant",
            "Electro Wizard",
            "Phoenix",
            "Sparky"
        ]
    },
    "Elixir Golem": {
        "counters": [
            "Tesla",
            "Barbarians",
            "Minion Horde",
            "Bomb Tower",
            "Wizard",
            "Barbarian Hut",
            "Three Musketeers",
            "Sparky",
            "Mega Knight"
        ]
    },
    "Battle Healer": {
        "counters": [
            "Barbarians",
            "Minion Horde",
            "Elite Barbarians",
            "Royal Recruits",
            "Rocket",
            "Battle Healer",
            "Inferno Tower",
            "Barbarian Hut",
            "Skeleton Army",
            "Dark Prince",
            "Hunter",
            "Witch",
            "Prince",
            "Bowler",
            "P.E.K.K.A",
            "Sparky",
            "Mega Knight"
        ]
    },
    "Skeleton King": {
        "counters": [
            "Bats",
            "Minions",
            "Arrows",
            "Valkyrie",
            "Poison",
            "Bowler",
            "Executioner",
            "Mega Knight"
        ]
    },
    "Archer Queen": {
        "counters": [
            "Royal Recruits",
            "Rocket",
            "Skeleton Army",
            "Void",
            "Lightning",
            "P.E.K.K.A",
            "Mega Knight",
            "Monk"
        ]
    },
    "Golden Knight": {
        "counters": [
            "P.E.K.K.A",
            "Inferno Dragon",
            "Mega Knight"
        ]
    },
    "Monk": {
        "counters": [
            "Barbarians",
            "Inferno Tower",
            "Skeleton Army",
            "Inferno Dragon"
        ]
    },
    "Skeleton Dragons": {
        "counters": [
            "Royal Delivery",
            "Skeleton Dragons",
            "Mega Minion",
            "Fireball",
            "Musketeer",
            "Wizard",
            "Rocket",
            "Baby Dragon",
            "Poison",
            "Lightning",
            "Inferno Dragon",
            "Archer Queen"
        ]
    },
    "Mother Witch": {
        "counters": [
            "Mega Minion",
            "Fireball",
            "Void",
            "Poison",
            "Lightning",
            "Mega Knight",
            "Miner"
        ]
    },
    "Electro Spirit": {
        "counters": [
            "Arrows",
            "Royal Delivery",
            "Fireball",
            "Barbarian Barrel",
            "The Log"
        ]
    },
    "Electro Giant": {
        "counters": [
            "Barbarians",
            "Mini P.E.K.K.A",
            "Barbarian Hut",
            "P.E.K.K.A",
            "Sparky"
        ]
    },
    "Phoenix": {
        "counters": [
            "Firecracker",
            "Dart Goblin",
            "Musketeer",
            "Flying Machine",
            "Zappies",
            "Inferno Tower",
            "Witch",
            "Electro Wizard",
            "Archer Queen"
        ]
    },
    "Little Prince": {
        "counters": [
            "Void",
            "Poison",
            "Electro Dragon",
            "Bowler",
            "Lightning",
            "P.E.K.K.A"
        ]
    },
    "Goblin Demolisher": {
        "counters": [
            "Tesla",
            "Fireball",
            "Flying Machine",
            "Void",
            "Bowler",
            "P.E.K.K.A",
            "Lightning",
            "Sparky",
            "Mega Knight"
        ]
    },
    "Goblin Machine": {
        "counters": [
            "Barbarians",
            "Elite Barbarians",
            "Mini P.E.K.K.A",
            "Goblin Cage",
            "Inferno Tower",
            "Prince",
            "P.E.K.K.A",
            "Fisherman",
            "Inferno Dragon",
            "Mighty Miner"
        ]
    },
    "Goblinstein": {
        "counters": [
            "Cannon",
            "Tesla",
            "Poison",
            "Fisherman"
        ]
    },
    "Rune Giant": {
        "counters": [
            "Inferno Tower",
            "P.E.K.K.A"
        ]
    },
    "Cannon": {
        "counters": [
            "Skeleton Barrel",
            "Earthquake",
            "Fireball",
            "Void",
            "Poison",
            "Lightning",
            "Princess",
            "Magic Archer"
        ]
    },
    "Goblin Hut": {
        "counters": [
            "Firecracker",
            "Earthquake",
            "Goblin Demolisher",
            "Rocket",
            "Lightning",
            "X-Bow"
        ]
    },
    "Mortar": {
        "counters": [
            "Earthquake",
            "Rocket",
            "Barbarian Hut",
            "Bowler",
            "Cannon Cart",
            "Lightning"
        ]
    },
    "Inferno Tower": {
        "counters": [
            "Zap",
            "Skeleton Barrel",
            "Minion Horde",
            "Earthquake",
            "Lightning",
            "Electro Wizard"
        ]
    },
    "Bomb Tower": {
        "counters": [
            "Earthquake",
            "Lightning"
        ]
    },
    "Barbarian Hut": {
        "counters": [
            "Earthquake",
            "Goblin Demolisher",
            "Lightning",
            "Magic Archer"
        ]
    },
    "Tesla": {
        "counters": [
            "Earthquake",
            "Cannon Cart",
            "Lightning"
        ]
    },
    "X-Bow": {
        "counters": [
            "Earthquake",
            "Rocket",
            "Barbarian Hut",
            "Bowler",
            "Lightning"
        ]
    },
    "Tombstone": {
        "counters": [
            "Arrows",
            "Earthquake",
            "Fireball",
            "Poison",
            "X-Bow",
            "The Log"
        ]
    },
    "Furnace": {
        "counters": [
            "Earthquake",
            "Flying Machine",
            "Rocket",
            "Poison",
            "Lightning"
        ]
    },
    "Goblin Cage": {
        "counters": [
            "Ice Golem",
            "Sparky",
            "Earthquake",
            "Lightning",
            "Princess"
        ]
    },
    "Goblin Drill": {
        "counters": [
            "Bomber",
            "Goblin Gang",
            "Arrows",
            "Skeleton Dragons",
            "Rascals",
            "Royal Recruits",
            "Valkyrie",
            "Goblin Demolisher",
            "Wizard",
            "Skeleton Army",
            "Dark Prince",
            "Bowler",
            "Royal Ghost",
            "Mega Knight"
        ]
    },
    "Goblin Barrel": {
        "counters": [
            "Arrows",
            "Goblin Gang",
            "Royal Delivery",
            "Fireball",
            "Barbarian Barrel",
            "Skeleton Army",
            "Tornado",
            "Freeze",
            "Bowler",
            "The Log"
        ]
    },
    "Graveyard": {
        "counters": [
            "Goblins",
            "Bats",
            "Berserker",
            "Minions",
            "Archers",
            "Goblin Gang",
            "Skeleton Dragons",
            "Barbarians",
            "Minion Horde",
            "Dart Goblin",
            "Valkyrie",
            "Goblin Curse",
            "Guards",
            "Skeleton Army",
            "Poison",
            "Witch",
            "Executioner",
            "Ice Wizard",
            "Electro Wizard",
            "Mother Witch"
        ]
    },
    "Clone": {
        "counters": [
            "Electro Spirit",
            "Arrows",
            "Tornado",
            "Poison",
            "Executioner"
        ]
    },
    "Heal Spirit": {
        "counters": [
            "Arrows",
            "Royal Delivery",
            "Fireball",
            "Barbarian Barrel",
            "The Log"
        ]
    }
}

Vamos a guardar la información de los counters:

In [34]:
with open("../data/raw/card_counters.json", "w", encoding="utf-8") as file:
    json.dump(card_counters, file, ensure_ascii=False, indent=4)

<a id="section4_2"></a>
### <font color="#00586D"> 4.2. Partidas</font>

Para las partidas, seguiremos la metodología explicada anteriormente.

El primer paso es obtener clanes con un número considerable de miembros, para después almacenar los *tags* de los jugadores y poder acceder al registro de batalla de cada uno.

Las siguientes funciones nos permiten hacer esto a partir de las ubicaciones y una serie de nombres aleatorios:

In [ ]:
def generate_random_clans(searches=100, seed=42):
    random.seed(seed)
    clan_tags = []
    location_ids = [location["id"] for location in get_locations()["items"]]
    for _ in range(searches):
        min_members = random.randint(40, 50)
        random_location = random.choice(location_ids)
        random_name = random.choice([
            "Dragon", "Shadow", "Mystic", "Royal", "Phoenix", "Thunder", 
            "Eternal", "Dark", "Golden", "Silver", "Inferno", "Storm", 
            "Crystal", "Flame", "Frost", "Emerald", "Titan", "Blaze", 
            "Glory", "Victory", "Chaos", "Valor", "Legacy", "Destiny", 
            "Harmony", "Justice", "Raven", "Wolf", "Lion", "Falcon", 
            "Eagle", "Knight", "Warrior", "Guardian", "Champion", "Vortex", 
            "Galaxy", "Nebula", "Comet", "Star", "Lunar", "Solar", "Cosmic", 
            "Blizzard", "Aurora", "Tempest", "Zephyr", "Echo", "Pulse", 
            "Radiant", "Shimmer", "Dawn", "Dusk", "Twilight", "Eclipse"
        ])
        clans = get_clans({"minMembers": min_members, "location": random_location, "name": random_name})["items"]
        clan_tags.extend([clan["tag"] for clan in clans])
    unique_clan_tags = list(set(clan_tags))
    return unique_clan_tags

In [ ]:
def get_player_tags(clan_tags):
    player_tags = [player["tag"] for clan_tag in clan_tags for player in get_clan_members(clan_tag)["items"]]
    return player_tags

Ahora las ultilizamos para obtener todos los jugadores a partir de los que extraremos las partidas de nuestro conjunto de datos:

In [48]:
clan_tags = generate_random_clans()
player_tags = get_player_tags(clan_tags)

Una vez tenemos los *tags* de los jugadores, es el momento de almacenar las partidas. Para ello, comprobaremos que el jugador tiene partidas recientes y almacenaremos la última sólo si pertenece al modo de juego que nos interesa.

In [50]:
def get_ladder_battles(player_tags):
    battles = []
    for player in player_tags:
        battle_log = get_player_battle_log(player)
        if battle_log:
            last_battle = battle_log[0]
            if last_battle["gameMode"]["name"] == "Ladder":
                battles.append(last_battle)
    return battles

In [ ]:
battles = get_ladder_battles(player_tags)

In [ ]:
with open("../data/raw/battles.json", "w", encoding="utf-8") as file:
    json.dump(battles, file, ensure_ascii=False, indent=4)

---

<a id="section5"></a>
## <font color="#00586D"> 5. Conclusiones</font>

A lo largo de esta fase se han obtenido datos sobre cartas y partidas de Clash Royale, los cuales serán utilizados posteriormente para crear un *dataset* que nos permita analizar la información y desarrollar modelos de aprendizaje automático capaces de predecir el resultado de nuevos enfrentamientos.

* Se ha utilizado la API oficial de Clash Royale para extraer información de cartas y partidas.

* La información de las cartas se ha complementado con propiedades adicionales extraidas de otras fuentes (tipos, counters...).

* Nos hemos enfocado exclusivamente en partidas del modo de juego *Camino de Trofeos*.

* Se ha evaluado la posibilidad complementar las partidas con datos del perfil de los jugadores (experiencia, récord de trofeos, maestrías de cartas...). Aunque en un uso real sí tendríamos acceso al perfil antes de predecir, no es seguro usar datos actuales para partidas pasadas durante el entrenamiento. Los perfiles cambian y este desfase temporal introduce sesgo, por lo que finalmente se rechazó esta opción.

Como futura mejora, se podría implementar un *datalake* o similar para almacenar partidas sin filtros y automatizar el proceso de adquisición de datos. De este modo, dispondríamos continuamente de información actualizada para poder llevar a cabo diferentes tareas de análisis de datos y *Machine Learning* utilizando partidas de los diferentes modos de juego de Clash Royale.

---